# Time domain methods for the curlcurl eigenvalue problem

### by M. Wess, 2024
*This Notebook is part of the `dualcellspaces` [documentation](https://ngsolve.github.io/dcm) for the addon package implementing the Dual Cell method in [NGSolve](https://ngsolve.org), as well as part of the `td_evp` [documentation](https://markuswess.github.io/td_evp/) on the implementation of time-domain methods for resonance problems.* 

In this notebook we show how to numerically solve the Maxwell eigenvalue problem combining `dualcellspaces` package with the ideas from

[1] L. Nannen and M. Wess, *A Krylov Eigenvalue Solver Based on Filtered Time Domain Solutions*, arXiv prepreint, 2024, [https://doi.org/10.48550/arXiv.2402.08515](https://doi.org/10.48550/arXiv.2402.08515).

We want to approximate the eigenvectors and eigenfunctions of the problem to find $\tilde\omega\in\mathbb R, E\in H(\mathrm{curl})(\Omega)$ for some domain $\Omega\subset\mathbb R^3$ such that

$$
\mathrm{curl}\mathrm{curl}E(x) = \tilde\omega^2 E(x)
$$
with natural boundary conditions.



## The filtered problem

The idea from [[1]](https://doi.org/10.48550/arXiv.2402.08515) in short is to use the solution of the time-domain Maxwell problem 

$$
\begin{aligned}
\partial_t^2 e(t,x) + \mathrm{curl}\mathrm{curl}e(t,x)&=0\\
e(0,x)&=E_0(x)
\end{aligned}
$$
To define the filtered operator $\tilde \Pi_\alpha$ by
$$
(\tilde \Pi_{\alpha} E^0)(x) = \int_0^\infty e(t,x)\alpha(t)dt
$$
for some weight function $\alpha$ with final support in $\mathbb R$.

It can be shown that for an eigenpair $(\tilde\omega, E)$ of the Maxwell eigenvalue problem $(\beta(\tilde\omega),E)$ is an eigenpair of $\Pi_\alpha$ for 

$$
\tilde \beta(s):=\int_0^\infty \cos(ts)\alpha(t)dt.
$$

## Discretizations

The same arguments hold true if we replace the continuous problem by the discrete eigenvalue problem

$$
\mathbf S \mathbf E = \omega^2\mathbf M \mathbf E
$$
where $\mathbf S,\mathbf M$ are the discretization of the $\mathrm {curl}\mathrm {curl}$ and the mass matrix and $\mathbf E$ is the coefficient vector of the discrete eigenfunctions.

Discretizing the semi-discrete time domain problem by a time-stepping and the integral by a trapezoidal rule leads to the discrete filter

$$
(\Pi_\alpha)(\mathbf{e^0})=\tau \sum_{j=0}^M \alpha(\tau j) \mathbf e^j
$$
where $\mathbf e^j$ is the time-domain solution at time step number $j$.

Then if $\omega,\mathbf E$  is an eigenpair of the discrete problem $(\beta(\omega),\mathbf E)$ is an eigenpair of the operator $\Pi_\alpha$ with
$$
\beta(s):= \tau \sum_{j=0}^M \lambda_j(s)\alpha(\tau j)
$$
and $\lambda_j$ is the approximation of the scalar problem
$$
\partial_t^2 l(t)+s^2 l(t)=0,\quad l(0) = 1
$$
by the chosen time-stepping.

### Choosing the filter and solving the problem

The idea is now to choose the weight function $\alpha$ such that the resulting filter $\beta$ is maximal in a region where the sought after eigenvalues are and close to zero otherwise. Then applying e.g., an Arnoldi algorithm to the filtered operator $\Pi_\alpha$ will yield approximations to the eigenfunctions where $\beta$ is maximal, which we know to be eigenvectors of the original problems corresponding to eigenvalues in the interesting region. Lastly we solve the original problem projected onto the space spanned by the Krylov space of $\Pi_\alpha$.

## Implementation

We start by doing the necessary imports

In [1]:
from ngsolve import *
import dualcellspaces as dcs
from ngsolve.webgui import Draw
import numpy as np
import matplotlib.pyplot as pl

We define the filtered operator as a class. Since we use a mixed method for the spacial discretization we need to supply the (inverse) mass matrices for the two fieds, as well as the discrete curl operator and the parameters for the time-discretization and filtering.

In [ ]:
class FilteredC(BaseMatrix):
    def __init__(self, matC, massE_inv, massH_inv, tau, weights, freedofsE = None, freedofsH = None):
        super().__init__()
        self.dt = tau
        self.weights = weights
        self.nsteps = len(weights)
        
        self.matC = matC
        self.massE_inv = massE_inv
        self.massH_inv = massH_inv
        
        self.freedofsE=freedofsE
        self.freedofsH=freedofsH
        
        
        self.vecE = self.massE_inv.CreateColVector()
        self.vecH = self.massH_inv.CreateColVector()
        self.tmpE = self.massE_inv.CreateColVector()
        self.tmpH = self.massH_inv.CreateColVector()


    def CreateColVector(self):
        return self.massE_inv.CreateColVector()
    
    def Shape(self):
        return self.massE_inv.shape
    
    def CreateVector(self,col):
        return self.massE_inv.CreateVector(col)
    
    def Mult(self,rhs,out):
        ######TODO
        with TaskManager():
            self.vecu.data = rhs
            tau = self.dt
            out.data = tau*weights[0]*self.vecu
            t = 0
            
            unew = self.tmpvec1
            uold = self.tmpvec2
            uold.data = self.vecu
            
            with TaskManager():
                for i in range(1,self.nsteps):
                    t += tau       
                    #print("\r time = {}, step = {}".format(t,i),end="")

                    unew.data = 2*self.vecu - uold 
                    unew.data -= tau**2 * self.matm_inv@self.mats * self.vecu
                    if self.freedofs:
                        unew.data[~self.freedofs] = 0.
                    uold.data = self.vecu
                    self.vecu.data = unew.data

                    out.data += tau*weights[i]*self.vecu